<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex10notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex10notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 過適合の抑制とモデル選択(3)
----

「ゴリゴリ君」のデータで，検証データを用いてモデルやハイパーパラメータの値を選択する実験をやってみよう．



まずはいつものように準備から．



In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# データを読み込む
dfGori = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/gorigori.csv', header=0)

これまでは，気温やアイス売上数の値をそのまま使っていましたが，今回は正規化して実験します．ただし，適当な数で割るだけという簡易的な方法です．

In [ ]:
# データを正規化
dfGori['気温'] /= 40
dfGori['アイス売上数'] /= 150

以下のセルは，実験用の関数の定義です．

In [ ]:
## 1, x, x^2, x^3, ..., X^D をならべたデータ行列（N x (D+1)）をつくる
#
def makeDataMatrix(x, D):

    N = x.shape[0]
    X = np.zeros((N, D+1))
    X[:, 0] = 1
    for i in range(1, D+1):
        X[:, i] = x**i

    return X

## nfold 分割交差検証用にデータを分割
#
def genDataset(X, Y, nfold, seed=0):

    N = X.shape[0]
    Nsub = N // nfold
    idx = np.empty(N, dtype=int)
    for n in range(nfold-1):
        idx[n*Nsub:(n+1)*Nsub] = n
    idx[(nfold-1)*Nsub:] = nfold-1
    np.random.seed(seed)
    np.random.shuffle(idx)

    XL = []
    YL = []
    XV = []
    YV = []
    for n in range(nfold):
        idx2 = idx != n
        XL.append(X[idx2, :])
        YL.append(Y[idx2])
        XV.append(X[~idx2, :])
        YV.append(Y[~idx2])

    return XL, YL, XV, YV

##  正規方程式を解く（正則化付き）
#
def solve(X, y, alpha=0.0):

    N = X.shape[0]
    A = np.dot(X.T, X)
    b = np.dot(X.T, y)
    if alpha > 0.0:
        Dp1 = X.shape[1]
        Id = np.eye(Dp1)
        Id[0, 0] = 0.0
        A += alpha * Id

    # x is the solution of the equation Ax = b
    x = np.linalg.solve(A, b)

    return x

以下で実験を行います．ここでは，次のような条件で多項式当てはめのモデルを学習させます．

- 多項式の次数: $D = 10$ と $D = 20$ の2通り
- 正則化項の係数: $\alpha = 0, 0.001, 0.01, 0.1$ の4通り

それぞれを組み合わせた8通りの条件で学習を行い，$n$-分割交差検証によって，汎化性能が最も高いと予想されるモデルを選ぶことにします．$n$ は以下のコード中の変数 `nfold` で指定しています．



In [ ]:
# データセットをいくつの組に分割するか
nfold = 3

msqeL = {}
msqeV = {}

# 多項式の次数 D 
for D in [10, 20]:

    # 学習用と検証用のデータを準備
    Xorg = makeDataMatrix(dfGori['気温'].to_numpy(), D)
    Yorg = dfGori['アイス売上数'].to_numpy()
    XL, YL, XV, YV = genDataset(Xorg, Yorg, nfold)

    # 正則化項の係数 alpha
    for alpha in [0.0, 0.001, 0.01, 0.1]:

        msqeLsub = np.empty(nfold)
        msqeVsub = np.empty(nfold)

        # nfold 組のデータのうち一つで学習と検証
        for n in range(nfold):
            # 正則化付き最小二乗法でD次多項式を当てはめ
            w = solve(XL[n], YL[n], alpha=alpha)
            # 学習データと検証データに対する平均二乗誤差
            msqeLsub[n] = np.mean(np.square(YL[n] - XL[n]@w))
            msqeVsub[n] = np.mean(np.square(YV[n] - XV[n]@w))

        # nfold 回の学習で得られた平均二乗誤差の平均を記録
        msqeL[(D, alpha)] = np.mean(msqeLsub)
        msqeV[(D, alpha)] = np.mean(msqeVsub)


上記では，8通りの条件のそれぞれについて，3通りの学習-検証データの組を使って学習と検証を行っています．
以下のセルを実行すると，それぞれの条件について，3-分割交差検証によって得られた，学習データに対する平均二乗誤差の平均（`msqeL`）と，検証データに対する平均二乗誤差の平均（`msqeV`）が表示されます．

In [ ]:
for key in msqeL.keys():
    D, alpha = key
    print(f'D = {D} alpha = {alpha}   msqeL = {msqeL[key]:.3e}   msqeV = {msqeV[key]:.3e}')

上記の結果を見て，これら8通りの条件のうち，汎化性能が最も高そうなものを選んで，その条件を紙にメモしておきましょう．

また，その条件を以下のセルの `D = ???` と `alpha = ???` のところに指定してセルを実行してみましょう．
このセルでは，指定した条件で，全ての学習データを用いて再度学習を行い，その結果を表示します．

In [ ]:
# すべてのデータを使って多項式当てはめ
D = ???
alpha = ???
Xorg = makeDataMatrix(dfGori['気温'].to_numpy(), D)
Yorg = dfGori['アイス売上数'].to_numpy()
w = solve(Xorg, Yorg, alpha=alpha)

# 曲線の式の値を計算
Xr =  np.linspace(0, 1, 100)
XXr = makeDataMatrix(Xr, D)
Yest = XXr @ w
print(Yest.shape)

# グラフを描く
fig, ax = plt.subplots(1, facecolor='white', figsize=(8, 6))
ax.scatter(Xorg[:, 1], Yorg)
ax.plot(Xr, Yest, color='red', label=f'D = {D}, $\\alpha$ = {alpha}')
ax.set_xlim(0.0, 1)
ax.set_ylim(0.0, 1)
ax.legend()
plt.show()